# Ambulance service sanity checks

Can we use CCG instead of LSOA?

The following figure shows how the CCGs have changed from 2015 to 2021. In 2015, the Isle of Wight had its own CCG. By 2021 it had been merged into Hampshire.

![The CCGs in Hampshire in the Isle of Wight. One subplot shows the CCGs from 2015, the other shows the CCGs from 2021.](figures/hants_ccg.png)


And the following figure picks out just Milton Keynes to show how it falls into the 2021 CCG and the ambulance service regions:

![The Milton Keynes CCG shown in separate maps as either part of the "NHS Bedfordshire, Luton and Milton Keynes CCG" or as part of the "South Central Ambulance Service" region.](figures/ambo_milton_keynes.png)


In [2]:
# Import required packages
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import geopandas as gpd

# # Linting
# %load_ext pycodestyle_magic
# %pycodestyle_on

In [336]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    data = './data'
    look_ups = 'look_ups'

    lsoa_to_ccg = ('Lower_Layer_Super_Output_Area_(2011)_to_Clinical_' +
                   'Commissioning_Group_to_Local_Authority_District_(April_' +
                   '2021)_Lookup_in_England.csv')
    lsoa_to_ccg15 = ('Lower_Layer_Super_Output_Area_(2011)_to_' +
                     'Clinical_Commissioning_Group_to_' +
                     'Local_Authority_District_(July_2015)_Lookup_in_England.csv')
    lsoa_to_sicbl22 = ('LSOA_(2011)_to_Sub_ICB_Locations_' +
                       '(July_2022)_Lookup_in_England.csv')

    lsoa_ambo_results = 'lsoa_ambo_lookup.csv'
    
    ccg15_amb_lookup = 'ccg15_amb.csv'
    ccg21_amb_lookup = 'ccg21_amb.csv'
    sicbl22_amb_lookup = 'sicbl22_amb.csv'

    ambo_name_dict = 'ambulance_service_names.csv'


paths = Paths()

## LSOA ambulance boundaries

In [7]:
lsoa_amb_data = pd.read_csv(os.path.join(paths.data, paths.lsoa_ambo_results))

lsoa_amb_data.head()

,LSOA11CD,ambulance_service,ambulance_service_name,LSOA11NM
0,E01000001,LAS,London,City of London 001A
1,E01000002,LAS,London,City of London 001B
2,E01000003,LAS,London,City of London 001C
3,E01000005,LAS,London,City of London 001E
4,E01000006,LAS,London,Barking and Dagenham 016A


## Compare catchment areas with CCG 2015

Can we instead assign ambulance service areas to CCG 2015 regions and get the same results as the LSOA regions?

__Create data__

Import LSOA-CCG15 lookup:

In [14]:
df_lsoa_ccg15 = pd.read_csv(os.path.join(paths.data, paths.look_ups, paths.lsoa_to_ccg15))

In [15]:
df_lsoa_ccg15.head()

,LSOA11CD,LSOA11NM,CCG15CD,CCG15CDH,CCG15NM,LAD15CD,LAD15NM,FID
0,E01010638,Bradford 004A,E38000001,02N,"NHS Airedale, Wharfedale and Craven CCG",E08000032,Bradford,1
1,E01023975,Ashford 007B,E38000002,09C,NHS Ashford CCG,E07000105,Ashford,2
2,E01010639,Bradford 004B,E38000001,02N,"NHS Airedale, Wharfedale and Craven CCG",E08000032,Bradford,3
3,E01023976,Ashford 008A,E38000002,09C,NHS Ashford CCG,E07000105,Ashford,4
4,E01023977,Ashford 007C,E38000002,09C,NHS Ashford CCG,E07000105,Ashford,5


Import a CCG15-ambulance service lookup that we've invented:

In [16]:
# Build ambulance service shapes from CCG 2015:
df_ccg15_ambo = pd.read_csv(os.path.join(paths.data, paths.ccg15_amb_lookup))

df_ccg15_ambo

,ccg15nm,ambulance_service_code
0,NHS Basildon and Brentwood CCG,EEAST
1,NHS Bedfordshire CCG,EEAST
2,NHS Cambridgeshire and Peterborough CCG,EEAST
3,NHS Castle Point and Rochford CCG,EEAST
4,NHS East and North Hertfordshire CCG,EEAST
...,...,...
204,NHS Rotherham CCG,YAS
205,NHS Scarborough and Ryedale CCG,YAS
206,NHS Sheffield CCG,YAS
207,NHS Vale of York CCG,YAS


Create a dataframe containing all LSOA and the ambulance service from the CCG2015 lookup:

In [17]:
df_lsoa_ccg15_ambo = pd.merge(
    df_lsoa_ccg15, df_ccg15_ambo,
    left_on='CCG15NM', right_on='ccg15nm', how='left'
)

Create a dataframe containing all LSOA, the ambulance service data from the LSOA, and the ambulance service data that we've just created using CCG 2015.

In [18]:
df_lsoa_compare = pd.merge(
    lsoa_amb_data[['LSOA11NM', 'LSOA11CD', 'ambulance_service']],
    df_lsoa_ccg15_ambo[['LSOA11CD', 'ambulance_service_code']],
    on='LSOA11CD', how='right'
)

In [19]:
df_lsoa_compare.head()

,LSOA11NM,LSOA11CD,ambulance_service,ambulance_service_code
0,Bradford 004A,E01010638,YAS,YAS
1,Ashford 007B,E01023975,SECAmb,SECAmb
2,Bradford 004B,E01010639,YAS,YAS
3,Ashford 008A,E01023976,SECAmb,SECAmb
4,Ashford 007C,E01023977,SECAmb,SECAmb


Do the ambulance service columns differ for any LSOA?

In [345]:
mask = df_lsoa_compare['ambulance_service'] == df_lsoa_compare['ambulance_service_code']

In [346]:
mask.all()

True

The two columns are always the same, so we can use CCG 2015 to define the ambulance service boundaries!